In [4]:
import utils.satellite_util
import matplotlib.pyplot as plt
import h5py
import numpy as np
from keras.layers import Dense
from keras.utils import plot_model

In [5]:
f = h5py.File('data/assorted_images/satellite_images.h5','r')
print('Keys: %s' % f.keys())
a_group_key = list(f.keys())[0]

Keys: KeysView(<HDF5 file "satellite_images.h5" (mode r)>)


In [6]:
data_original = np.array(f['data'])
labels = np.array(f['labels'])
print(data_original)

import utils.satellite_util as ut
def download_all_images(zoom=16, width=224, height=224):
    coordinates = ut.get_coordinate_list(200)
    ## YOUR CODE HERE
    for i in range(len(coordinates)):
        lat = coordinates[i][0]
        long = coordinates[i][1]
        utils.satellite_util.download_img(lat, long, save = True)
    pass

visualize_data = download_all_images()

[[[[ 47  63  53]
   [ 46  62  52]
   [ 48  64  54]
   ...
   [ 52  68  58]
   [ 49  65  55]
   [ 46  62  52]]

  [[ 49  65  55]
   [ 48  64  54]
   [ 50  66  56]
   ...
   [ 49  65  55]
   [ 49  65  55]
   [ 48  64  54]]

  [[ 47  62  55]
   [ 47  62  55]
   [ 49  64  57]
   ...
   [ 46  62  52]
   [ 49  65  55]
   [ 51  67  57]]

  ...

  [[ 58  56  57]
   [ 56  54  55]
   [ 56  56  56]
   ...
   [ 49  61  57]
   [ 53  65  61]
   [ 46  58  54]]

  [[ 57  53  54]
   [ 56  54  55]
   [ 57  57  57]
   ...
   [ 49  61  57]
   [ 52  64  60]
   [ 45  57  53]]

  [[ 53  49  50]
   [ 56  54  55]
   [ 61  59  60]
   ...
   [ 46  58  54]
   [ 48  60  56]
   [ 41  53  49]]]


 [[[110 127 121]
   [109 126 120]
   [115 127 123]
   ...
   [135 131 132]
   [139 133 135]
   [138 132 134]]

  [[116 133 127]
   [117 132 127]
   [122 134 130]
   ...
   [132 130 131]
   [138 134 135]
   [138 134 135]]

  [[122 137 132]
   [123 138 133]
   [130 142 138]
   ...
   [128 130 129]
   [133 133 133]
   [134 134

HTTPError: HTTP Error 403: Forbidden

In [7]:
train_ratio, val_ratio = 0.9, 0.05
X_train = data_original[:int(data_original.shape[0]*train_ratio), ...] # ... means all the other axes
y_train = labels[:int(data_original.shape[0]*train_ratio), ...]

X_val = data_original[int(data_original.shape[0]*train_ratio):int(data_original.shape[0]*(train_ratio+val_ratio)), ...]
y_val = labels[int(data_original.shape[0]*train_ratio):int(data_original.shape[0]*(train_ratio+val_ratio)), ...]

X_test = data_original[int(data_original.shape[0]*(train_ratio+val_ratio)):, ...]
y_test = labels[int(data_original.shape[0]*(train_ratio+val_ratio)):, ...]

print(X_train.shape)
print(X_test.shape)
print(X_val.shape)

(576, 224, 224, 3)
(33, 224, 224, 3)
(32, 224, 224, 3)


In [8]:
import keras
from keras.models import Sequential # Sequential is one of the main models in Keras, which is basically a sequentially stacked series of layers
from keras.callbacks import EarlyStopping

In [9]:
early_stopping = EarlyStopping(patience=50)
#model = create_model()

#learning rate = [10**-3, 3*10**-3, 10**-2,3*10**-2,0.1,0.3,1,3,10]
learning_rate = [10**-3, 3*10**-3, 10**-2,3*10**-2]
#learning_rate = [10**-6]
layers = [2,3,4,5]
CNN_layers = [2,3]
#layers = [4]
#CNN_layers = [3]
#neurons = [32]
neurons = [8,16,32,64]
# First we'll use fully-connected neural nets
#from keras.layers import Dense # Dense is Keras's name for fully connected layers


for rate in learning_rate:
    for num in layers:
        for n in neurons:
            for c in CNN_layers:
                print(rate,num,n,c)
                model = Sequential() # Re-initialize the model
                for i in range(c):
                    model.add(keras.layers.Conv2D(filters=32, kernel_size=5, strides=(2, 2), padding='valid'))
                    model.add(keras.layers.Activation('relu'))
                    model.add(keras.layers.MaxPooling2D(pool_size=(2, 2))) # By default the stride is the same as the pooling size

                model.add(keras.layers.Flatten())
            
                for i in range(num-1):
                    model.add(keras.layers.Dense(n))
                    model.add(keras.layers.Activation('relu'))

                model.add(keras.layers.Dense(1))
                model.add(keras.layers.Activation('sigmoid'))

# Compilation
                model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(lr=0.0000001),metrics = ['accuracy']) #, 'f1'

# Training
                history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val), shuffle = True, callbacks=[early_stopping])
            #print('\n',history.history.keys())

# Evaluation
                acc = model.evaluate(X_test, y_test)
                print('The test accuracy is: {}'.format(acc))
                ac1 = model.predict()
# And make predictions
                prob = model.predict(X_test) # These are probabilities, and we want to convert them to class labels
                #print(prob)
                label = np.array(prob > 0.5, dtype=int)
            
            #evaluate_acc(model)
                print('The predicted probabilities are: {}'.format(prob))
                print('The predicted class labels are: {}'.format(label))
                plt.plot(history.history['loss'])
                plt.plot(history.history['val_loss'])
                plt.title('Model_loss')
                plt.ylabel('loss')
                plt.xlabel('Epoch')
                plt.legend(['Train','Val'],loc='upper left')
                plt.show()
                plt.plot(history.history['acc'])
                plt.plot(history.history['val_acc'])
                plt.title('Accuracy')
                plt.ylabel('acc')
                plt.xlabel('Epoch')
                plt.legend(['Train','Val'],loc='upper left')
                plt.show()

0.001 2 8 2
Train on 576 samples, validate on 32 samples
Epoch 1/20
576/576 [==============================] - 7s 12ms/step - loss: 3.0976 - acc: 0.7847 - val_loss: 2.9295 - val_acc: 0.8125
Epoch 2/20
576/576 [==============================] - 7s 11ms/step - loss: 3.0866 - acc: 0.7847 - val_loss: 2.9244 - val_acc: 0.8125
Epoch 3/20
576/576 [==============================] - 8s 14ms/step - loss: 3.0750 - acc: 0.7847 - val_loss: 2.9192 - val_acc: 0.8125
Epoch 4/20
576/576 [==============================] - 8s 14ms/step - loss: 3.0625 - acc: 0.7847 - val_loss: 2.9133 - val_acc: 0.8125
Epoch 5/20
576/576 [==============================] - 8s 14ms/step - loss: 3.0502 - acc: 0.7847 - val_loss: 2.9050 - val_acc: 0.8125
Epoch 6/20
576/576 [==============================] - 9s 15ms/step - loss: 3.0353 - acc: 0.7847 - val_loss: 2.8972 - val_acc: 0.8125
Epoch 7/20
576/576 [==============================] - 9s 15ms/step - loss: 3.0207 - acc: 0.7847 - val_loss: 2.8893 - val_acc: 0.8125
Epoch 8/20
5

TypeError: predict() missing 1 required positional argument: 'x'

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model_loss')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.legend(['Train','Val'],loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model_loss')
plt.ylabel('acc')
plt.xlabel('Epoch')
plt.legend(['Train','Val'],loc='upper left')
plt.show()